In [4]:
import importlib
import sys
sys.path.append('../src')

import qa
importlib.reload(qa)
import qa
from qa import QA

import pickle

In [2]:
# from datasets import load_dataset

# ds = load_dataset("TIGER-Lab/MMLU-Pro")

# mmlu = {}
# for ele in ds['test']:
#     if ele['category'] not in mmlu:
#         mmlu[ele['category']] = []
#     mmlu[ele['category']].append(ele)

# import random

# mmlu_samples = []
# for cat in mmlu:
#     mmlu_samples.extend(random.Random(0).sample(mmlu[cat], 10))

# with open('../data/mmlu_samples_140.pkl', 'wb') as f:
#     pickle.dump(mmlu_samples, f)

with open('../data/mmlu_samples_140.pkl', 'rb') as f:
    mmlu_samples = pickle.load(f)

In [ ]:
with open('../data/res_mmlu_org.pkl', 'rb') as f:
    res_mmlu_org = pickle.load(f)


for agree_size in [0,1,2,3,4]:
    for disagree_size in [0,1,2,3,4]:
        eval_feat = {
            'type': 'grp',
            'agree_size': agree_size,
            'disagree_size': disagree_size,
            'disagree_type': '2nd',
        }

        res_mmlu_grp_2nd = qa.mmlu_eval(res_mmlu_org, eval_feat)


Processing MMLU: 100%|██████████| 140/140 [00:00<00:00, 1205754.74it/s]

Eval Type:grp, #Agree: 0, #Disagree: 0, Disagree Type: 2nd, Accuracy: 0.457



Processing MMLU: 100%|██████████| 140/140 [00:00<00:00, 1007208.51it/s]

Eval Type:grp, #Agree: 0, #Disagree: 1, Disagree Type: 2nd, Accuracy: 0.386



Processing MMLU: 100%|██████████| 140/140 [00:00<00:00, 583700.36it/s]

...

Eval Type:grp, #Agree: 4, #Disagree: 2, Disagree Type: 2nd, Accuracy: 0.457



Processing MMLU: 100%|██████████| 140/140 [00:00<00:00, 1075462.56it/s]

Eval Type:grp, #Agree: 4, #Disagree: 3, Disagree Type: 2nd, Accuracy: 0.45



Processing MMLU: 100%|██████████| 140/140 [00:00<00:00, 2053155.80it/s]

Eval Type:grp, #Agree: 4, #Disagree: 4, Disagree Type: 2nd, Accuracy: 0.457
